# Часть 1. Исследование датасета и обучение модели
## Задачи
1. Исследование датасета
2. Предобработка данных (пропуски, дубликаты)
3. Исследовательский анализ данных (выбросы, взаимосвязи между признаками)
4. Обучение модели
5. Подготовка предсказания на тестовой выборке

## Подготовительные шаги

In [ ]:
# контроль версий
!pip install -q numpy==1.22.4 pandas==1.5.1
!pip install -q -U scipy
!pip install -q matplotlib==3.5
!pip install -q -U seaborn
!pip install -q -U phik
!pip install -q -U shap
!pip install -q -U scikit-learn

In [ ]:
# загрузка библиотек
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import shap
import seaborn as sns
from scipy import stats
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer
)
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# константы
RANDOM_STATE = 42

In [ ]:
# настройки отображения датафреймов
pd.set_option('display.max_columns', None)

## Загрузка и предварительное изучение данных

In [ ]:
# загрузка датасетов
train = pd.read_csv('heart_train.csv', index_col='id', usecols=lambda x: x not in ['Unnamed: 0'])
test = pd.read_csv('heart_test.csv', index_col='id', usecols=lambda x: x not in ['Unnamed: 0'])

In [ ]:
# функция для вывода информации о данных
def df_info(df, df_name):
    print("=" * 60)
    # вывод информации о датафрейме
    print('\033[1m' + f'Информация о датафрейме {df_name}:' + '\033[0m')
    df.info()
    print("-" * 60)
    # вывод 5 первых строк
    print('\033[1m' + f'Первые 5 строк датафрейма {df_name}:' + '\033[0m')
    display(df.head())

In [ ]:
# информация о датафреймах
df_info(train, 'train')
df_info(test, 'test')

### Описание данных
- Age -- возраст
- Cholesterol -- холестерин
- Heart rate -- частота сердечных сокращений
- Diabetes -- наличие диабета
- Family History -- семейный анамнез
- Smoking -- курение
- Obesity -- ожирение
- Alcohol Consumption -- употребление алкоголя
- Exercise Hours Per Week -- часы тренировок в неделю
- Diet -- диета
- Previous Heart Problems -- наличие предыдущих проблем с сердцем
- Medication Use -- принимает ли препараты
- Stress Level -- уровень стресса
- Sedentary Hours Per Day -- часы сидячего образа жизни в день
- Income -- доход
- BMI -- индекс мыссы тела
- Triglycerides -- триглицериды 
- Physical Activity Days Per Week -- дни физической активности в неделю
- Sleep Hours Per Day -- часы сна в день
- Heart Attack Risk (Binary) -- риск сердечного приступа
- Blood sugar -- уровень сахара в крови
- CK-MB -- креатинфосфокиназа-МВ
- Troponin -- тропонин
- Gender -- пол
- Systolic blood pressure -- систолическое артериальное давление
- Diastolic blood pressure -- диастолическое артериальное давление

### Изучение признаков в данных

In [ ]:
# функция для выделения признаков
def features(df):
    return df.columns.tolist()

In [ ]:
# функция для выделения категориальных признаков
def catcols(df):
    catcols = []
    for col in features(df):
        if df[col].dtype == 'O' or len(df[col].unique()) <= 4:
            catcols.append(col)
    return catcols

In [ ]:
# функция для выделения количественных признаков
def numcols(df):
    numcols = list(filter(lambda item: item not in catcols(df), df.select_dtypes(include=[np.number]).columns.tolist()))
    return numcols

In [ ]:
print('\033[1m' + 'Категориальные признаки в train:' + '\033[0m')
catcols(train)

In [ ]:
print('\033[1m' + 'Количественные признаки в train:' + '\033[0m')
numcols(train)

In [ ]:
# проверка, что эти списки вместе дают общий список признаков
set(catcols(train) + numcols(train)) == set(features(train))

### Выводы
- В данных встречаются пропуски
- Количественные данные в большинстве приведены к единой шкале (от 0 до 1), то есть, они были предварительно подвергнуты MinMax масштабированию. Не масштабированы признаки Stress Level и Physical Activity Days Per Week
- Некоторые категориальные и количественные признаки, кодируемые целыми числами, имеют неверный тип признака (float). Исправим это в тренировочной выборке, а в тестовой обработаем внутри пайплайна

In [ ]:
# список с признаками для изменения типов
cols_to_convert = ['Diabetes',
                   'Family History',
                   'Smoking',
                   'Obesity',
                   'Alcohol Consumption',
                   'Previous Heart Problems',
                   'Medication Use',
                   'Stress Level',
                   'Physical Activity Days Per Week',
                   'Heart Attack Risk (Binary)']

In [ ]:
train_init = train.copy()

In [ ]:
# замена типов данных в train
for col in cols_to_convert:
    train[col] = train[col].astype('Int64')

print('Типы данных в train после конвертации:')
train.dtypes

## Предобработка тренировочных данных

### Пропуски

In [ ]:
# подсчет пропусков в train
print('\033[1m' + 'Пропуски в train:' + '\033[0m')
train.isna().sum()

Пропуски обнаружились в нескольких признаках, причем в одном и том же количестве строк. Проверим, встречаются ли эти пропуски в одном и том же сабсете (для примера возьмем сабсет с пропусками в Diabetes):

In [ ]:
# подсчет пропусков в сабсете train
print('\033[1m' + 'Пропуски в сабсете train с пропусками в признаке Diabetes:' + '\033[0m')
train[train['Diabetes'].isna()].isna().sum()

Действительно, все пропуски встречаются в одном сабсете. Вероятно, эти данные собирались в одном месте или одновременно, и часть данных не была получена. 

In [ ]:
# подсчет пропусков в test
print('\033[1m' + 'Пропуски в test:' + '\033[0m')
test.isna().sum()

In [ ]:
# подсчет пропусков в сабсете test
print('\033[1m' + 'Пропуски в сабсете test с пропусками в признаке Diabetes:' + '\033[0m')
test[test['Diabetes'].isna()].isna().sum()

В тестовой выборке ситуация аналогична.

#### Сравнение сабсетов с пропусками и без
Изучим эту группу пациентов внимательнее и сопоставим ее с остальными пациентами.

Изучим распределения признаков, в которых данные не пропущены, для двух сабсетов -- с полными данными (train_full) и с пропусками в данных (train_missing).

In [ ]:
# выделяем сабсеты
train_missing = train[train['Diabetes'].isna()]
train_full = train[~train['Diabetes'].isna()]

In [ ]:
# список признаков с пропущенными значениями
print('\033[1m' + 'Признаки с пропущенными значениями:' + '\033[0m')
features_missing = train.isna().sum()[train.isna().sum() > 0].index.tolist()
features_missing

In [ ]:
# выделяем релевантные количественные и категориальные признаки
features_full = list(filter(lambda item: item not in features_missing, features(train)))

numcols_full = train[features_full].select_dtypes(include='float64').columns.tolist()  
catcols_full = train[features_full].select_dtypes(include=['Int64', 'object']).columns.tolist()

print('\033[1m' + 'Количественные признаки без пропусков:' + '\033[0m')
print(numcols_full)
print(60*'-')
print('\033[1m' + 'Категориальные признаки без пропусков:' + '\033[0m')
print(catcols_full)

In [ ]:
# функция для построения графиков для количественных признаков
def hist_bar(df, feature, ax):    
    if len(df[feature].unique()) <= 20:
        sns.countplot(ax=ax, data=df, x=col, color='skyblue', edgecolor='black', alpha=0.7)
        ax.tick_params(axis='x', rotation=45)
    else:
        sns.histplot(ax=ax, data=df, x=col, color='skyblue', edgecolor='black', alpha=0.7)

In [ ]:
# функция для построения графиков для категориальных признаков
def piechart(df, feature, ax):
    category_counts = df[feature].value_counts()
    ax.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=90)

In [ ]:
# графики для количественных признаков
sns.set(font_scale=3)

for col in numcols_full:
    fig, axes = plt.subplots(1, 2, figsize=(60, 20), gridspec_kw={'width_ratios': [1, 1]})
    fig.suptitle(f'Распределение признака {col}', fontsize=40)
    
    axes[0].set_title('В train_full', size=40)
    hist_bar(train_full, col, axes[0])
    
    axes[1].set_title('В train_missing', size=40)
    hist_bar(train_missing, col, axes[1])
    
    plt.show()

In [ ]:
# графики для категориальных признаков
for col in catcols_full:
    fig, axes = plt.subplots(1, 2, figsize=(60, 20), gridspec_kw={'width_ratios': [1, 1]})
    fig.suptitle(f'Распределение признака {col}', fontsize=40)
    
    axes[0].set_title('В train_full', size=40)
    piechart(train_full, col, axes[0])
    
    axes[1].set_title('В train_missing', size=40)
    piechart(train_missing, col, axes[1])
    
    plt.show()
sns.set(font_scale=1)

#### Выводы
- Обнаружилось, что все пропущенные данные касаются людей с определенной категорией диеты (закодированной как 3), в то время как в остальных данных представлены люди с тремя другими категориями диеты.
- Данные этой группы с большой вероятностью привенесут шум в модель и не будут информативны:
    - В них нет пациентов с риском сердечного приступа
    - Некоторые признаки в них представлены лишь одним значением
- Исходя из этого, данные с пропусками лучше исключить из рассмотрения. В таком случае, модель может давать для пациентов с категорией диеты 3 в тестовой выборке несколько смещенные предсказания. Поскольку эта категория пациентов в целом наименее представлена, и для нее большая часть признаков отсутствует или малоинформативна, это представляется не таким критичным. Необходимо будет учитывать, что у модели есть это ограничение.

In [ ]:
print('\033[1m' + 'Количество пациентов с различными диетами в test:' + '\033[0m')
test.reset_index().pivot_table(index='Diet', values='id', aggfunc='count')

Пациенты с диетой 3 составляют меньшинство и в тестовой выборке.

In [ ]:
print('\033[1m' + 'Уникальные значения признаков у пациентов с диетой 3 в test:' + '\033[0m')
for f in features(test):
    print('\033[1m' + f + '\033[0m')
    print(test[test['Diet']==3][f].unique())
    print(60*'-')

В тестовой выборке у пациентов с диетой 3 также многие признаки неинформативным (представлены единственным значением). Также важно запомнить, что пол в этой группе закодирован по-другому (вероятно, 1 - м, 0 - ж, учитывая распределение признака и имеющиеся конвенции).

In [ ]:
print('Данные с пропусками удалены из train.')
print('\033[1m' + 'Проверка (подсчет пропусков в train):' + '\033[0m')
train = train[~train['Diabetes'].isna()]
train.isna().sum()

In [ ]:
print('\033[1m' + 'Уникальные значения Diet в train:' + '\033[0m')
train['Diet'].unique().tolist()

### Дубликаты

#### Полные дубликаты
Проверим наличие полных дубликатов:

In [ ]:
# подсчет полных дубликатов
print('\033[1m' + 'Полные дубликаты в train:' + '\033[0m')
train.duplicated().sum()

In [ ]:
# подсчет полных дубликатов
print('\033[1m' + 'Полные дубликаты в test:' + '\033[0m')
test.duplicated().sum()

Полных дубликатов нет.

#### Неявные дубликаты
Проверим наличие неявных дубликатов, выведя уникальные значения:

In [ ]:
# вывод уникальных значений
print('\033[1m' + 'Уникальные значения признаков в train:' + '\033[0m')
for col in features(train):
    print('\033[1m' + col + '\033[0m')
    print(train[col].unique())
    print(60*'-')

Неявных дубликатов внутри тренировочной выборки не обнаружилось.

In [ ]:
# вывод уникальных значений
print('\033[1m' + 'Уникальные значения признаков в test:' + '\033[0m')
for col in features(test):
    print('\033[1m' + col + '\033[0m')
    print(test[col].unique())
    print(60*'-')

In [ ]:
for diet in test['Diet'].unique().tolist():
    print(f'Кодирование пола в данных пациентов с диетой {diet}')
    print(test.loc[test['Diet'] == diet, 'Gender'].unique().tolist())

Неявных дубликатов в тестовой выборке не обнаружилось, но важно отметить, что пол в тестовых данных закодирован по-разному в зависимости от типа диеты. Аналогично было и в тренировочных данных, но мы исключили сабсет с диетой 3.

#### Выводы
В данных не обнаружилось дубликатов

## Исследовательский анализ данных

На этом этапе изучим наличие выбросов в признаках, а также взаимосвязи между входными и целевым признаком в тренировочной и тестовой выборках.

### Количественные признаки в train

In [ ]:
# функция для вывода графиков
def graphs(df, col):    
   # выведем гистограммы и диаграммы размаха
    # задаем сетку с графиками
    fig, axes = plt.subplots(1, 2, figsize=(10, 5), gridspec_kw={'width_ratios': [3, 1]})
    fig.suptitle(f'Распределение признака {col}')
    
    # гистограмма
    axes[0].hist(df[col], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0].axvline(x=df[col].median(), color='blue', linestyle='--', label='Медиана')
    axes[0].grid(True, linestyle='--', alpha=0.5)
    axes[0].legend()
    
    # диаграмма размаха
    axes[1].boxplot(df[col])
    axes[1].set_xticks([])
    axes[1].set_xticklabels([])
    axes[1].grid(True, linestyle='--', alpha=0.5)
    
    # отображение графиков
    plt.tight_layout()
    plt.show()

    print(90*'-')

In [ ]:
# графики для количественных признаков
print('\033[1m' + 'Графики для количественных признаков в train:' + '\033[0m')
for col in numcols(train):
    graphs(train, col)

#### Выводы
В тренировочной выборке обнаружились выбросы в Blood sugar, Troponin и CK-MB. Похоже, что в них большинство значений попадает в очень узкий промежуток. Проверим это при помощи сводных таблиц:

In [ ]:
for col in ['Blood sugar', 'Troponin', 'CK-MB']:
    print(f'Количество наблюдений с уникальными значениями {col}')
    print(train.reset_index().pivot_table(index=col, values='id', aggfunc='count').sort_values(by='id', ascending=False))

Во всех трех признаках подавляющее число наблюдений -- это одно и то же значение. Вероятнее всего, эти признаки окажутся не слишком информативными. Удалим наиболее выбивающиеся данные:

In [ ]:
# удаление выбросов из train
print('Доля исключенных наблюдений:')
len_init = train.shape[0]
train = train[(train['Blood sugar'] <= 0.5) & (train['Troponin'] <= 0.4) & (train['CK-MB'] <= 0.4)]
round(1 - train.shape[0]/len_init, 2)

In [ ]:
for col in ['Blood sugar', 'Troponin', 'CK-MB']:
    graphs(train, col)

### Количественные признаки в test

In [ ]:
# графики для количественных признаков
print('\033[1m' + 'Графики для количественных признаков в test:' + '\033[0m')
for col in numcols(test):
    if test[col].isna().sum() > 0:
        graphs(test.dropna(), col)
    else:
        graphs(test, col)

#### Выводы
В test обнаружился выброс в heart rate. Также обнаружились выбросы в Blood suger, Troponin и CK-MB, как и в тренировочных данных. В остальном распределения схожи, поскольку все признаки были масштабированы.

In [ ]:
for col in ['Blood sugar', 'Troponin', 'CK-MB']:
    print(f'Количество наблюдений с уникальными значениями {col}')
    print(test.reset_index().pivot_table(index=col, values='id', aggfunc='count').sort_values(by='id', ascending=False))

Для Blood sugar, Troponin и CK-MB ситуация в test аналогична: большая часть наблюдений -- это одно и то же значение.

### Категориальные признаки в train и test

In [ ]:
# графики для категориальных признаков
sns.set(font_scale=3)
for col in catcols(test):
    fig, axes = plt.subplots(1, 2, figsize=(60, 20), gridspec_kw={'width_ratios': [1, 1]})
    fig.suptitle(f'Распределение признака {col}', fontsize=40)
    
    axes[0].set_title('В train', size=40)
    piechart(train, col, axes[0])
    
    axes[1].set_title('В test', size=40)
    piechart(test, col, axes[1])
    
    plt.show()

sns.set(font_scale=1)

#### Выводы
Категориальные признаки в train и test распределены схоже, только в test сохранены пациенты с диетой 3 и пол закодирован двумя разными способами.

### Матрицы корреляций для train и test

Чтобы различие в кодировании пола не повлияло на подсчет корреляций, возьмем для test сабсет с Diet не равной 3.

In [ ]:
interval_cols = ['Age',
                 'Cholesterol',
                 'Heart rate',
                 'Exercise Hours Per Week',
                 'Sedentary Hours Per Day',
                 'Income',
                 'BMI',
                 'Triglycerides',
                 'Sleep Hours Per Day',
                 'Blood sugar',
                 'CK-MB',
                 'Troponin',
                 'Systolic blood pressure',
                 'Diastolic blood pressure']

fig, axes = plt.subplots(1, 2, figsize=(120, 50), gridspec_kw={'width_ratios': [1, 1]})
fig.suptitle('Матрицы корреляций для тренировочной и тестовой выборок', fontsize=40)

sns.set(font_scale=2.5)

sns.heatmap(ax=axes[0], data=train[features(train)].phik_matrix(interval_cols=interval_cols),
            cmap='coolwarm', center=0, annot=True)
axes[0].set_title('Матрица корреляций для train', size=40)
axes[0].tick_params('x', labelsize=30, rotation=90)
axes[0].tick_params('y', labelsize=30, rotation=0)

test_subset = test[test['Diet'] != 3]
sns.heatmap(ax=axes[1], data=test_subset[features(test_subset)].phik_matrix(interval_cols=interval_cols),
            cmap='coolwarm', center=0, annot=True)
axes[1].set_title('Матрица корреляций для test', size=40)
axes[1].tick_params('x', labelsize=30, rotation=90)
axes[1].tick_params('y', labelsize=30, rotation=0)

plt.show()

sns.set(font_scale=1)

#### Выводы
- Матрицы корреляций для тренировочной и тестовой выборок схожи
- Большинство входных признаков очень слабо связаны с Heart attack risk. На этапе моделирования не будем включать признаки с корреляцией равной 0.

In [ ]:
# список признаков для исключения
features_to_exclude = ['Cholesterol',
                       'Heart rate',
                       'Family History',
                       'Smoking',
                       'Exercise Hours Per Week',
                       'Previous Heart Problems',
                       'Medication Use',
                       'Stress Level',
                       'Income',
                       'BMI',
                       'Triglycerides',
                       'Gender']

In [ ]:
print('Релевантные признаки для моделирования:')
features_relevant = list(filter(lambda x: x not in features_to_exclude and x != 'Heart Attack Risk (Binary)', features(train)))
features_relevant

In [ ]:
print('Релевантные категориальные признаки:')
catcols(train[features_relevant])

In [ ]:
print('Релевантные количественные признаки:')
numcols(train[features_relevant])

In [ ]:
cols_to_convert_relevant = list(filter(lambda x: x in features_relevant, cols_to_convert))
cols_to_convert_relevant

## Обучение модели и предсказание

Для предсказания риска сердечного приступа будем использовать пайплайн, в котором будут заданы предобработка признаков и моделирование. Переберем несколько моделей классификации, и с помощью кроссвалидации подберем лучшую модель на основании метрики ROC-AUC. Эта метрика оптимальна, поскольку в предсказании риска сердечного приступа для точности прогноза важно минимизировать и ложноположительные, и ложноотрицательные результаты.

#### Функции для моделирования

In [ ]:
# функция для задания выборок
def split(train, test, features, target):
    
    X_train = train[features]
    y_train = train[target]
    X_test = test[features]
    
    return X_train, X_test, y_train

In [ ]:
# функция для подбора модели
def pipeline_search(pipeline, param_grid, X_train, y_train):
    rs = RandomizedSearchCV(pipeline,
                            param_grid,
                            cv=5,
                            scoring='roc_auc',
                            error_score='raise',
                            n_jobs=-1,
                            n_iter=30,
                            random_state=RANDOM_STATE
                            )
    rs.fit(X_train, y_train)

    print('Лучшая модель:\n\n', rs.best_estimator_)
    print('Лучшие параметры:\n\n', rs.best_params_)
    print('Метрика лучшей модели на кросс-валидации:', rs.best_score_.round(3))
    
    return rs

In [ ]:
# функция для моделирования
def prediction(model, X_test):    
    return model.predict(X_test)

#### Составляющие пайплайна для предобработки данных
В рамках предобработки в пайплайне необходимо:
- Поменять типы данных для признаков из списка cols_to_convert
- Масштабировать те количественные признаки, к которым не был применен MinMax ранее (Physical Activity Days Per Week)
- Закодировать категориальные признаки при помощи OneHotEncoder (Diet) и OrdinalEncoder (Diabetes, Obesity, Alcohol Consumption)
- Заполнить пропуски медианой (в количественных признаках) и модой (в категориальных признаках)

In [ ]:
# класс для конвертации типов
class TypeConverter(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns=None, dtype='Int64'):
        self.columns = columns
        self.dtype = dtype
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        
        if self.columns == None:
            return X_transformed.astype(self.dtype)
        else:
            for col in self.columns:
                if col in X_transformed.columns:
                    X_transformed[col] = X_transformed[col].astype(self.dtype)
            return X_transformed

In [ ]:
# пайплайн для масштабирования и кодирования
# SimpleImputer + OneHotEncoder для неупорядоченных признаков
ohe_pipe = Pipeline(
    [
        (
            'simpleImputer_ohe', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ohe', 
            OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', sparse_output=False)
        )
    ]
)

# SimpleImputer + OrdinalEncoder для упорядоченных признаков
ord_pipe = Pipeline(
    [('simpleImputer_before_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ord',  OrdinalEncoder(categories=[[0, 1],
                                         [0, 1],
                                         [0, 1]],
                             handle_unknown='use_encoded_value', unknown_value=np.nan)),
     ('simpleImputer_after_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
    ]
)

# создаём общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ['Diet']),
     ('ord', ord_pipe, ['Diabetes', 'Obesity', 'Alcohol Consumption']),
     ('num_imputer', SimpleImputer(missing_values=np.nan, strategy='median'), ['Age',
                                                                               'Sedentary Hours Per Day',
                                                                               'Physical Activity Days Per Week',
                                                                               'Sleep Hours Per Day',
                                                                               'Blood sugar',
                                                                               'CK-MB',
                                                                               'Troponin',
                                                                               'Systolic blood pressure',
                                                                               'Diastolic blood pressure']),
     ('num_scaler', MinMaxScaler(), ['Physical Activity Days Per Week'])
    ], 
    remainder='passthrough'
)

#### Создание общего пайплайна

In [ ]:
# общий пайплайн: подготовка данных и модель
pipe_final = Pipeline([
    ('type_converter', TypeConverter(dtype='Int64', columns=['Diabetes',
                                                             'Obesity',
                                                             'Alcohol Consumption',
                                                             'Physical Activity Days Per Week'])),
    ('preprocessor', data_preprocessor),
    ('model', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

In [ ]:
# параметры
param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'model': [DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
        'model__max_depth': range(2,9)
    },
    
    # словарь для модели KNeighborsClassifier() 
    {
        'model': [KNeighborsClassifier()],
        'model__n_neighbors': range(2,20, 2)
    },

    # словарь для модели LogisticRegression()
    {
        'model': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', 
            penalty='l2',
            class_weight='balanced'
        )],
        'model__C': range(1,5)
    },
     # словарь для модели RandomForestClassifier()
    {
        'model':[RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced')],
        'model__n_estimators':range(2,100, 2)
    }
]

#### Моделирование

In [ ]:
X_train, X_test, y_train = split(train, test, features_relevant, 'Heart Attack Risk (Binary)')

In [ ]:
rs = pipeline_search(pipe_final, param_grid, X_train, y_train)

In [ ]:
y_pred = prediction(rs, X_test)

#### Выводы
Лучшая модель предсказывает сердечный приступ с ROC AUC = 0.54. Это может быть связано с тем, что в данных связи входных признаков с целевым были очень слабыми.

## Сохранение предсказаний

In [ ]:
# создание датафрейма
ids = pd.DataFrame(X_test.index)
preds = pd.DataFrame(y_pred)
predictions = ids.join(preds)
predictions.columns = ['id', 'prediction'] 
predictions.head(10)

In [ ]:
# сохранение
predictions.to_csv('predictions.csv', sep=',', encoding='utf-8', index=False)

## Сохранение модели

In [ ]:
# сохраняем лучшую модель
joblib.dump(rs, 'heart_model.pkl')